In [48]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import evaluate, print_perf, GridSearch


random.seed(561)

In [3]:
os.getcwd()
os.chdir('D:\\Columbia University\\Personalization\\Project\\lastfm-dataset-1K')

In [4]:
users = pd.read_table('userid-profile.tsv', header=0)
data = pd.read_table('userid-timestamp-artid-artname-traid-traname.tsv',
                     header=-1,
                     #nrows=20000000,
                     error_bad_lines=False)

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


In [5]:
data = data.rename(columns={0:'userid', 1:'timestamp', 2:'artistid', 3:'artistname', 4:'trackid', 5:'trackname'})

data['timestamp'] = pd.to_datetime(data['timestamp'])

For the mini-project, we are using a smaller dataset. The following transformations will convert the dataset to use number of plays as our metric, grouped by user and artist.

To help with our data cleaning and setting up the matrices, we used [this website](https://jessesw.com/Rec-System/) to guide us.

In [6]:
data = data.groupby(['userid', 'artistname']).size().reset_index(name='plays')

In [5]:
users = list(np.sort(data.userid.unique())) # Get our unique users
artists = list(data.artistname.unique()) # Get our unique artists
quantity = list(data.plays) # All of our plays

rows = data.userid.astype('category', categories = users).cat.codes 
# Get the associated row indices
cols = data.artistname.astype('category', categories = artists).cat.codes 
# Get the associated column indices
plays_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(users), len(quantity)))
plays_sparse

<992x897419 sparse matrix of type '<class 'numpy.int64'>'
	with 897419 stored elements in Compressed Sparse Row format>

In [6]:
# Sparsity of the matrix
matrix_size = plays_sparse.shape[0]*plays_sparse.shape[1] # Number of possible interactions in the matrix
num_plays = len(plays_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_plays/matrix_size))
sparsity

99.8991935483871

## Dataset Reduction

The sparsity of 99.899% is extremely sparse, even for matrices that are intended to be sparse. We experimented with removing all rare artists, which had minimal effect on the sparsity. Instead, we will include only the top 100 artists, which has some improvement on the sparsity.

In [8]:
rare_artists = data.query("plays < 6"). \
    groupby('artistname').size().reset_index(name='users_listening_to_artist'). \
    query("users_listening_to_artist < 10")
    
top100_artists = data.groupby('artistname')['plays'].sum().reset_index(name='plays'). \
    nlargest(100,'plays')

In [9]:
reduced_data = data[~data.artistname.isin(rare_artists['artistname'])]

print(reduced_data.shape, data.shape)

(596817, 3) (897419, 3)


In [10]:
users = list(np.sort(reduced_data.userid.unique())) # Get our unique users
artists = list(reduced_data.artistname.unique()) # Get our unique artists
quantity = list(reduced_data.plays) # All of our plays

rows = reduced_data.userid.astype('category', categories = users).cat.codes 
# Get the associated row indices
cols = reduced_data.artistname.astype('category', categories = artists).cat.codes 
# Get the associated column indices
plays_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(users), len(quantity)))

plays_sparse

<992x596817 sparse matrix of type '<class 'numpy.int64'>'
	with 596817 stored elements in Compressed Sparse Row format>

In [11]:
# Sparsity of the matrix
matrix_size = plays_sparse.shape[0]*plays_sparse.shape[1] # Number of possible interactions in the matrix
num_plays = len(plays_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_plays/matrix_size))
sparsity

99.8991935483871

While 99.898% sparsity is not great, it is an improvement over the previous matrix, and so we will use this to set up our brute force model.

For SVD, we use the `surprise` package.

In [36]:
usertotal = data.groupby('userid')['plays'].sum().reset_index(name="total_plays")
normalized_data = pd.merge(reduced_data, usertotal)
normalized_data['normalized_plays'] = normalized_data['plays']/normalized_data['total_plays']
normalized_data.drop(['total_plays'], inplace=True, axis=1)

In [39]:
#normalized_data[(normalized_data.plays > 0)]['plays'] = 1
normalized_data.loc[normalized_data['plays'] != 0, 'plays'] = 1

In [50]:
normalized_data.head()

,userid,artistname,plays,normalized_plays
0,user_000001,4 Wings,1,0.00012
1,user_000001,4Hero,1,0.00875
2,user_000001,A New Funky Generation,1,0.00012
3,user_000001,A Reminiscent Drive,1,0.00006
4,user_000001,A Taste Of Honey,1,0.00030


In [44]:
reader = Reader(rating_scale=(0, 1))

# The columns must correspond to user id, item id and ratings (in that order).
model_data = Dataset.load_from_df(normalized_data[['userid', 'artistname', 'plays']], reader)

model_data.split(n_folds=3)

# We'll use the famous SVD algorithm.
algo = SVD(reg_all=.1)

# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, model_data, measures=['RMSE', 'MAE'])

print_perf(perf)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.0038
MAE:  0.0010
------------
Fold 2
RMSE: 0.0037
MAE:  0.0010
------------
Fold 3
RMSE: 0.0038
MAE:  0.0010
------------
------------
Mean RMSE: 0.0037
Mean MAE : 0.0010
------------
------------
        Fold 1  Fold 2  Fold 3  Mean    
RMSE    0.0038  0.0037  0.0038  0.0037  
MAE     0.0010  0.0010  0.0010  0.0010  


In [49]:
param_grid = {'n_epochs': [5: 30], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
grid_search = GridSearch(SVD, param_grid, measures=['RMSE', 'MAE'])

model_data = Dataset.load_from_df(normalized_data[['userid', 'artistname', 'plays']], reader)
model_data.split(n_folds=3)

grid_search.evaluate(model_data)

results_df = pd.DataFrame.from_dict(grid_search.cv_results)

[{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}, {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}, {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}, {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}, {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}, {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}, {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}, {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}]
------------
Parameters combination 1 of 8
params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}
------------
Mean RMSE: 0.0121
Mean MAE : 0.0043
------------
------------
Parameters combination 2 of 8
params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}
------------
Mean RMSE: 0.0075
Mean MAE : 0.0022
------------
------------
Parameters combination 3 of 8
params:  {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}
------------
Mean RMSE: 0.0038
Mean MAE : 0.0009
------------
------------
Parameters combination 4 of 8
params:  {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}
-----

In [51]:
results_df

,MAE,RMSE,lr_all,n_epochs,params,scores
0,0.004344,0.012078,0.002,5,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}","{'RMSE': 0.0120779607043, 'MAE': 0.00434387681..."
1,0.002249,0.007503,0.002,5,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}","{'RMSE': 0.00750293536731, 'MAE': 0.0022491665..."
2,0.000948,0.003760,0.005,5,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}","{'RMSE': 0.00376007020641, 'MAE': 0.0009475713..."
3,0.000430,0.002097,0.005,5,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}","{'RMSE': 0.00209664770608, 'MAE': 0.0004304878..."
4,0.001401,0.005225,0.002,10,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}","{'RMSE': 0.00522464728907, 'MAE': 0.0014014317..."
5,0.000630,0.002854,0.002,10,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}","{'RMSE': 0.00285391164887, 'MAE': 0.0006297201..."
6,0.000260,0.001338,0.005,10,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}","{'RMSE': 0.00133815920903, 'MAE': 0.0002598039..."
7,0.000120,0.000688,0.005,10,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}","{'RMSE': 0.000687904558719, 'MAE': 0.000119600..."


In [45]:
reader = Reader(rating_scale=(0, 1))

# The columns must correspond to user id, item id and ratings (in that order).
model_data = Dataset.load_from_df(normalized_data[['userid', 'artistname', 'normalized_plays']], reader)

model_data.split(n_folds=3)

# We'll use the famous SVD algorithm.
algo = SVD(reg_all=.1)

# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, model_data, measures=['RMSE', 'MAE'])

print_perf(perf)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.0071
MAE:  0.0024
------------
Fold 2
RMSE: 0.0072
MAE:  0.0023
------------
Fold 3
RMSE: 0.0072
MAE:  0.0024
------------
------------
Mean RMSE: 0.0072
Mean MAE : 0.0024
------------
------------
        Fold 1  Fold 2  Fold 3  Mean    
RMSE    0.0071  0.0072  0.0072  0.0072  
MAE     0.0024  0.0023  0.0024  0.0024  


In [ ]:
# THEORY: is the model just guessing very low values? Adjust the penalty to remove this possibility